In [1]:
import feasible_test as ft
import feasible_test_with_print_error as fterr
import random
import functions as func
import functions_with_print as funcpt
import pandas as pd
import numpy as np

#讀取courseDetail
courseDetail = pd.read_csv('data/course.csv')[['course code', 'Number of students', 'instructor']]
courseDetail['course code']=courseDetail['course code'].astype(str)
RoomDetail = pd.read_csv('data/classroom.csv')[['classroom', 'cr_capacity']]

#variables
roomNum=3 #教室數量
weekdays=5 #上課日子
dailyParts=3 #parts=將一天劃分為[早上、下午、晚上]
period=weekdays*dailyParts #15 一個weekdays中，不分教室的區塊總數
session=roomNum*weekdays*dailyParts #45 一個weekdays中，空教室的總數(一維陣列的長度)
k=weekdays*roomNum #15 [早上、下午、晚上] 一個part中的session數(索引調整參數)
totalCourseNum=30

def get_score(schedule):
    return func.ObjFun(schedule,courseDetail, roomNum, k, RoomDetail, session, period, totalCourseNum)

64.47452300785633
rule1:  55.24691358024691
[2, 1, 2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 2, 2, 0]
68 60.0 75.0
rule2:  89.33333333333333
rule3:  30.42424242424242
rule4:  36.0
Objval: 56.320426487093144
56.320426487093144


In [2]:
def cross(schedule_1,schedule_2,cross_index1):
    # select cross index
#     cross_index1 = random.randint(0, len(schedule_1)-1)
#     cross_index2 = random.randint(0, len(schedule_2)-1)
#     while(cross_index1 >= cross_index2):
#         cross_index1 = random.randint(0, len(schedule_1)-1)
#         cross_index2 = random.randint(0, len(schedule_2)-1)
#     cross_index1 = 2
#     cross_index1 = random.randint(0, len(schedule_1)-1-6)
    cross_index2 = cross_index1 + 6

    
#     print("({},{})".format(cross_index1,cross_index2))

    # make new schedule
    new_schedule = schedule_1[0: cross_index1] + schedule_2[cross_index1 : cross_index2] + schedule_1[cross_index2 : 45]   
#     print(new_schedule)
    
    # origin
    # 2 8 | 6 4 5 | 7 1 3
    # 8 7 | 2 1 3 | 4 6 5
    
    # cross finish
    # 2 8 | 2 1 3 | 7 1 3
    # 8 7 | 6 4 5 | 4 6 5 
    
    # fix (ex: 2去換回"自己"原本有的6)
    # 6 8 | 2 1 3 | 7 4 5
    # 8 7 | 6 4 5 | 4 6 5
    
    for i in range(cross_index1,cross_index2): #cross範圍
        data = new_schedule[i] # i位置資料 (ex:此處是2)
        data_index = schedule_1.index(data) #此筆資料位於原schedule的什麼位置 (ex:此處2找到位於為schedule "0的位置" )
#         print(data_index)
        new_schedule[data_index] = schedule_1[i] # new_schedule的該位置換成cross對應字串 (ex:此處原"0的位置" 2換成 "i對應位置的6" )
#         print(new_schedule[data_index])

    return new_schedule
  
    

In [15]:
def one_generation(schedule1,schedule2):
# print(ft.feasible_test(temp_schedule))
# print(ft.feasible_test(temp_schedule2))
#     result = False
    stop_flag= False
    gene_pool = []
    gene_score = []

#  cross_index1 = random.randint(0, len(schedule_1)-1-6)
    for i in range(len(schedule1)-6):
#     while(result == False):
        new_schedule = cross(schedule1,schedule2,i)
    #     print(new_schedule)
        result = ft.feasible_test(new_schedule)    
        if result == True:
    #         print("第{}次 : 找到feasible解!".format(i))
    #         print(get_score(new_schedule))
            gene_pool.append(new_schedule)
            gene_score.append(float(get_score(new_schedule)))


        new_schedule2 = cross(schedule2,schedule1,i)
        result = ft.feasible_test(new_schedule2)
        if result == True:
    #         print("第{}次 : 找到feasible解!".format(i))
    #         print(get_score(new_schedule))
            gene_pool.append(new_schedule2)
            gene_score.append(float(get_score(new_schedule)))

    #     print(new_schedule)
    #     print(new_schedule2)
    
#     print(gene_score)
    gene_score_sorted = sorted(gene_score, reverse=True)
#     print(gene_score_sorted)
    next_gen1 = gene_pool[gene_score.index(gene_score_sorted[0])]
    next_gen2 = gene_pool[gene_score.index(gene_score_sorted[1])]
    if max(gene_score_sorted) == min( gene_score_sorted):
        stop_flag = True
    return next_gen1,next_gen2,stop_flag



In [16]:
temp_schedule = ['306000001','','306008001','356461001','','306016002','356425001','','306016012','307873001','','307867001','','','307870001',
 '306016022','307857001','306050011','','356388001','307851001','306525001','','307035001','','356395001','307034001','356564001','306736001','306737001',
'307932001','','356822001','','356389001','356019001','','356387001','307901001','356813001','','356808001','','','307942001']

temp_schedule2 = ['306008001', '307942001', '', '306525001', '356808001', '', '307851001', '306736001', '', '307867001', '307873001', '', '356019001', '356388001', '', '356387001', '356395001', '', '356813001', '306016022', '', '306000001', '307034001', '', '306737001', '356389001', '', '307857001', '306016002', '', '356425001', '307901001', '', '356461001', '307035001', '', '356822001', '307932001', '', '306016012', '356564001', '', '307870001', '306050011', '']

best_score = 0
best_schedule= []
stop_flag = False

print("-----初始解-----")
print(ft.feasible_test(temp_schedule))
if get_score(temp_schedule) > best_score:
    best_score = get_score(temp_schedule)
    best_schedule = temp_schedule
    
print(ft.feasible_test(temp_schedule2))
if get_score(temp_schedule2) > best_score:
    best_score = get_score(temp_schedule2)
    best_schedule = temp_schedule2

next_gen1,next_gen2 = temp_schedule,temp_schedule2

for i in range(1,100):
    print("-----第{}子代-----".format(i))
    next_gen1,next_gen2,stop_flag = one_generation(next_gen1,next_gen2)
    if stop_flag:
        break
        
    if get_score(next_gen1) > best_score:
        best_score = get_score(next_gen1)
        best_schedule = next_gen1
        
    
    
print("-----已找到最佳解，得分為：{}-----".format(best_score))    
print(best_schedule)

    # print(ft.feasible_test(next_gen1))
    # print(next_gen1)
    # print(ft.feasible_test(next_gen2))
    # print(next_gen2)
# print("-----第2子代-----")
# next_gen1,next_gen2 = one_generation(next_gen1,next_gen2)
# print("-----第3子代-----")
# next_gen1,next_gen2 = one_generation(next_gen1,next_gen2)
# print("-----第4子代-----")
# next_gen1,next_gen2 = one_generation(next_gen1,next_gen2)
# print("-----第5子代-----")
# next_gen1,next_gen2 = one_generation(next_gen1,next_gen2)
# print("-----第6子代-----")
# next_gen1,next_gen2 = one_generation(next_gen1,next_gen2)
# print("-----第7子代-----")
# next_gen1,next_gen2 = one_generation(next_gen1,next_gen2)
# print("-----第8子代-----")
# next_gen1,next_gen2 = one_generation(next_gen1,next_gen2)
# print("-----第9子代-----")
# next_gen1,next_gen2 = one_generation(next_gen1,next_gen2)
# print("-----第10子代-----")
# next_gen1,next_gen2 = one_generation(next_gen1,next_gen2)

-----初始解-----
True
67.99382716049382
67.99382716049382
True
73.18383838383838
73.18383838383838
-----第1子代-----
68.79953558574249
67.70606060606062
68.15050505050506
68.47382638647008
67.0511320097527
68.2305661983823
69.31795735129069
-----第2子代-----
69.37856341189674
69.37856341189674
69.89708193041527
69.89708193041527
69.89708193041527
69.89708193041527
73.14643755563296
73.14643755563296
73.14643755563296
71.06038159371492
69.31795735129069
69.31795735129069
69.31795735129069
69.31795735129069
69.31795735129069
69.31795735129069
69.31795735129069
69.31795735129069
69.31795735129069
69.31795735129069
68.18462401795736
68.18462401795736
66.74253647586981
66.74253647586981
67.51757420952825
67.51757420952825
68.91548821548821
71.03041526374861
71.47485970819304
67.76610549943884
67.76610549943884
71.99513139053369
71.03216842757072
71.03216842757072
71.03216842757072
68.66509539842873
68.66509539842873
68.66509539842873
68.66509539842873
66.76700336700337
-----第3子代-----
66.767003367003

In [29]:
a = [2,8,6,4,5,7,1,3]
b = [8,7,2,1,3,4,6,5]

# origin
# 2 8 | 6 4 5 | 7 1 3
# 8 7 | 2 1 3 | 4 6 5

# cross finish
# 2 8 | 2 1 3 | 7 1 3
# 8 7 | 6 4 5 | 4 6 5 

# fix (ex: 2去換回"自己"原本有的6)
# 6 8 | 2 1 3 | 7 4 5
# 8 7 | 6 4 5 | 4 6 5

print(a)
result = cross(a,b)
print(result)

[2, 8, 6, 4, 5, 7, 1, 3]
(0,12)


IndexError: list index out of range

In [30]:
import random
random.randint(0, 1)


1

In [21]:
mystr = "happy"
mystr[0:2]+","+mystr[2:4]+","+mystr[4:5]
mystr[0:2]+","+mystr[2:4]+","+mystr[5:5]

'ha,pp,'

In [24]:
for i in range(32):
    print(i,end=",")

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,

In [ ]:
for i in range(2, 4):
    print(mystr[i])

In [3]:
schedule_feasible = ['306000001','','306008001','356461001','','306016002','356425001','','306016012','307873001','','307867001','','','307870001',
 '306016022','307857001','306050011','','356388001','307851001','306525001','','307035001','','356395001','307034001','356564001','306736001','306737001',
'307932001','','356822001','','356389001','356019001','','356387001','307901001','356813001','','356808001','','','307942001']

schedule_feasible2 = ['','','306008001','306016002','306000001','','356425001','','306016012','307873001','','307867001','307942001','','307870001',
 '306016022','307857001','306050011','356387001','356388001','307851001','306525001','','307035001','306736001','356395001','307034001','356461001','','306737001',
'307932001','','356822001','','356389001','356019001','356564001','','307901001','356813001','','356808001','','','']

schedule_miss_class = ['','','306008001','','','306016002','356425001','','306016012','307873001','','307867001','307942001','','307870001',
 '306016022','307857001','306050011','356387001','356388001','307851001','306525001','','307035001','306736001','356395001','307034001','356461001','','306737001',
'307932001','','356822001','','356389001','356019001','356564001','','307901001','356813001','','356808001','','','']

schedule_multiple_profeesor = ['306000001','306000001','306008001','','','306016002','356425001','','306016012','307873001','','307867001','307942001','','307870001',
 '306016022','307857001','306050011','356387001','356388001','307851001','306525001','','307035001','306736001','356395001','307034001','356461001','','306737001',
'307932001','','356822001','','356389001','356019001','356564001','','307901001','356813001','','356808001','','','']

print(ft.feasible_test(schedule_feasible))
print(ft.feasible_test(schedule_miss_class))
print(ft.feasible_test(schedule_multiple_class))
print(ft.feasible_test(schedule_multiple_profeesor))

print(fterr.feasible_test(schedule_feasible))
print(fterr.feasible_test(schedule_miss_class))
print(fterr.feasible_test(schedule_multiple_class))
print(fterr.feasible_test(schedule_multiple_profeesor))

True
False
False
False
True
Error: 所有的課程應該都被排下去
False
Error: 不可以有重複課程
False
Error: 不可以有重複課程
False


In [5]:
test = ['306008001', '307942001', '', '306525001', '356808001', '', '307851001', '306736001', '', '307867001', '307873001', '', '356019001', '356388001', '', '356387001', '356395001', '', '356813001', '306016022', '', '306000001', '307034001', '', '306737001', '356389001', '', '307857001', '306016002', '', '356425001', '307901001', '', '356461001', '307035001', '', '356822001', '307932001', '', '306016012', '356564001', '', '307870001', '306050011', '']
test2 = ['306008001', '307942001', '', '306525001', '356808001', '', '307851001', '306736001', '', '307867001', '307873001', '', '356019001', '356388001', '', '356387001', '356395001', '', '356813001', '306016022', '', '306000001', '307034001', '', '306737001', '356389001', '', '307857001', '306016002', '', '356425001', '307901001', '', '356461001', '307035001', '', '356822001', '307932001', '', '306016012', '356564001', '', '307870001', '306050011', '']


print(fterr.feasible_test(test))
print(fterr.feasible_test(test2))

True
True
